# Calculate data for REZ regions

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:50:00'
cores = 12
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.54.52:8787/status,
Dashboard: http://10.6.54.52:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.54.52:45361,Workers: 0
Dashboard: http://10.6.54.52:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
# client.close()
# cluster.close()

In [6]:
# %load_ext autoreload
# %autoreload 2

In [7]:
import xarray as xr
import numpy as np

In [8]:
import functions as fn

# Load REZ mask

In [9]:
rez_mask = xr.open_dataset(
    '/g/data/w42/dr6273/work/projects/Aus_energy/data/rez_mask_era5_grid.nc'
).REZ

# Load gridded data

In [10]:
years = range(1959, 2021)

Mean solar shortwave radiation downwards

In [11]:
mssrd = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/msdwswrf/msdwswrf_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_REZ_region.zarr',
    consolidated=True
)

100m wind speed

In [12]:
w100 = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100w/100w_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_REZ_region.zarr',
    consolidated=True
)

# Calculate regional averages

In [34]:
rez_mask

<xarray.DataArray 'REZ' (region: 39, lat: 721, lon: 1440)>
[40491360 values with dtype=bool]
Coordinates:
  * lat          (lat) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon          (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * region       (region) object 'N1' 'N2' 'N3' 'N4' ... 'V3' 'V4' 'V5' 'V6'
    region_code  (region) int64 ...

In [32]:
def REZ_spatial_mean(da, region):
    region_mask = rez_mask.sel(region=region)
    return da.where(region_mask).mean(['lat', 'lon'])

In [ ]:
def _by_REZ(agg_function, da):
    da_list = []
    for region in rez_mask['region'].values:
        region_agg = agg_function

In [33]:
REZ_spatial_mean(w100['w100'], 'N1')

array(2.6580365, dtype=float32)

In [35]:
REZ_spatial_mean

<function __main__.REZ_spatial_mean(da, region)>